In [6]:
def viterbi(obs, 
            states, 
            start_p,
            observations,
            trans_p,
            emit_p):
    V = [{}]
    # We are going to iterate through all the observations,
    # starting from normal. 
    # Calculate the probability for the given states.
    for i, st in enumerate(states):
        V[0][st] = {'prob': start_p[st] * emit_p[st][obs[0]], 
                    'prev': None} 
        
    # For the rest of the observations.
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            # Take the first state and calculate the probability.
            # Use it as the initial guess.
            max_tr_prob = V[t-1][states[0]]['prob'] * trans_p[states[0]][st]
            prev_st_selected = states[0]

            # Now loop through the remaining states and compare the probability.
            # Take the one with the highest probability.
            for prev_st in states[1:]:
                tr_prob = V[t-1][prev_st]['prob'] * trans_p[prev_st][st]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st

            max_prob = max_tr_prob * emit_p[st][obs[t]]
            V[t][st] = {'prob': max_prob,
                        'prev': prev_st_selected}
    for line in dptable(V):
        print(line)

    opt = []
    # The highest probability.
    max_prob = max(value['prob']
                   for value in V[-1].values())
    max_prob
    # Get the most probable state and its backtrack.
    for st, data in V[-1].items():
        if data['prob'] == max_prob:
            opt.append(st)
            previous = st
            break
    # Follow the backtrack until the first observation. 
    # Minus 2, since we exclude the last result.
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t+1][previous]['prev'])
        previous = V[t+1][previous]['prev']
    print(f'The step of states are {" ".join(opt)} with the probability of {max_prob}')
    return V, [states.index(o) for o in opt]

In [7]:
def dptable(V):
    header = [f'{"":12s}'] + [f'{str(i):12s}' for i in range(len(V))]
    yield ' '.join(header)
    
    for state in V[0].keys():
        row = [f'{state:12s}'] + [f'{str(round(d[state]["prob"], 5)):12s}' 
                                  for d in V]
        yield ' '.join(row)

    yield ''
    for state in V[0].keys():
        row = [f'{state:12s}'] + [f'{str(d[state]["prev"]):12s}' 
                                  for d in V]
        yield ' '.join(row)

In [8]:
obs = ('normal', 'cold', 'dizzy')
states = ('Healthy', 'Fever')
start_p = {'Healthy': 0.6,
           'Fever': 0.4}
trans_p = {'Healthy': {'Healthy': 0.7, 'Fever': 0.3},
           'Fever': {'Healthy': 0.4, 'Fever': 0.6}}
emit_p = {'Healthy': {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
          'Fever': {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6}}

In [13]:
result, path = viterbi(obs, states, start_p, obs, trans_p, emit_p)
path

             0            1            2           
Healthy      0.3          0.084        0.00588     
Fever        0.04         0.027        0.01512     

Healthy      None         Healthy      Healthy     
Fever        None         Healthy      Healthy     
The step of states are Healthy Healthy Fever with the probability of 0.01512


[0, 0, 1]

In [14]:
newobs = [obs[o] for o in [0,1,2,1,2]]
print(newobs)
result, path = viterbi(newobs, states, start_p, obs, trans_p, emit_p)
path

['normal', 'cold', 'dizzy', 'cold', 'dizzy']
             0            1            2            3            4           
Healthy      0.3          0.084        0.00588      0.00242      0.00017     
Fever        0.04         0.027        0.01512      0.00272      0.00098     

Healthy      None         Healthy      Healthy      Fever        Healthy     
Fever        None         Healthy      Healthy      Fever        Fever       
The step of states are Healthy Healthy Fever Fever Fever with the probability of 0.000979776


[0, 0, 1, 1, 1]

In [15]:
newobs = [obs[o] for o in [0,2,1,2,1]]
print(newobs)
result, path = viterbi(newobs, states, start_p, obs, trans_p, emit_p)
path

['normal', 'dizzy', 'cold', 'dizzy', 'cold']
             0            1            2            3            4           
Healthy      0.3          0.021        0.00864      0.0006       0.00056     
Fever        0.04         0.054        0.00972      0.0035       0.00063     

Healthy      None         Healthy      Fever        Healthy      Fever       
Fever        None         Healthy      Fever        Fever        Fever       
The step of states are Healthy Fever Fever Fever Fever with the probability of 0.0006298559999999999


[0, 1, 1, 1, 1]

In [20]:
obs = ('sleeping', 'eating', 'pooping')

obs_map = [obs[i] for i in [1,1,2,1,0,1,2,1,0,2,2,0,1,0,1]]

states = ('Healthy', 'Fever')

start_p = {'Healthy': 0.5,
           'Fever': 0.5}
trans_p = {'Healthy': {'Healthy': 0.7, 'Fever': 0.3},
           'Fever': {'Healthy': 0.4, 'Fever': 0.6}}
emit_p = {'Healthy': {'sleeping': 0.2, 'eating': 0.6, 'pooping': 0.2},
          'Fever': {'sleeping': 0.4, 'eating': 0.1, 'pooping': 0.5}}

In [21]:
result, path = viterbi(obs_map, states, start_p, obs, trans_p, emit_p)
path

             0            1            2            3            4            5            6            7            8            9            10           11           12           13           14          
Healthy      0.3          0.126        0.01764      0.00741      0.00104      0.00044      6e-05        3e-05        0.0          0.0          0.0          0.0          0.0          0.0          0.0         
Fever        0.05         0.009        0.0189       0.00113      0.00089      5e-05        7e-05        0.0          0.0          0.0          0.0          0.0          0.0          0.0          0.0         

Healthy      None         Healthy      Healthy      Healthy      Healthy      Healthy      Healthy      Healthy      Healthy      Healthy      Fever        Fever        Fever        Healthy      Healthy     
Fever        None         Healthy      Healthy      Fever        Healthy      Fever        Healthy      Fever        Healthy      Fever        Fever        Fever      

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0]